# CryoTEMPO: Assessing Subglacial Lakes Dynamics in the Thwaites area, Westerm Antarctica

In this tutorial, we will use CryoTEMPO-EOLIS gridded product and CryoTEMPO Land Ice to study the evolution of the surface elevation of four subglacial lakes within the Thwaites glacier, in Western Antarctica.

### Environment Setup
To run this notebook, you will need to make sure that the folllowing packages are installed in your python environment (all can be installed via pip/conda):
- matplotlib
- geopandas
- contextily
- ipykernel
- shapely
- specklia

If you are using the Google Colab environment, these packages will be installed in the next cell. Please note this step may take a few minutes.

In [ ]:
import sys
if 'google.colab' in sys.modules:
    %pip install rasterio --no-binary rasterio
    %pip install specklia
    %pip install matplotlib
    %pip install geopandas
    %pip install contextily
    %pip install shapely
    %pip install python-dotenv

In [ ]:
import os
from time import perf_counter
from datetime import datetime, timedelta
import geopandas as gpd
import math
import pandas as pd
from shapely import Point

from typing import List, Tuple
import numpy as np
from numpy.typing import NDArray
from matplotlib import cm

import contextily as ctx
import matplotlib.pyplot as plt
from specklia import Specklia

# local EPSG projection code for plotting, we're considering data in the southern hemisphere so will use the
# Southern Polar Stereographic projection
sps_epsg_code = 3031

# To run this code yourself, first generate your own key using https://specklia.earthwave.co.uk.
if 'DEMO_API_KEY' in os.environ:
    client = Specklia(os.environ['DEMO_API_KEY'])
else:
    user_api_key = input('Please generate your own key using https://specklia.earthwave.co.uk/ApiKeys and paste it here:')
    client = Specklia(user_api_key)

## Download the file containing the subglacial lakes extent

The polygons describing the extent of the subglacial lakes are stored in a file on Google Drive. Its ID is "1kByOjALzLrxueYHHgSSrfgjRI_73B9tT".
Run the next cell to download this file. If you click on the "Files" icon in the left bar, you will see that now the file is present in the local storage associated with this Colab session.

In [ ]:
! gdown 1kByOjALzLrxueYHHgSSrfgjRI_73B9tT

## Load the file and plot the polygons on a map

NB: the lake extents are expressed in projected coordinates (EPSG:3031).

In [ ]:
lakes_extent_file_path = os.path.join(os.getcwd(), 'thwaites_sub_glacial_lake_mask.gpkg')
lakes_extent_df = gpd.read_file(lakes_extent_file_path)
lakes_extent_df

In [ ]:
fig, ax = plt.subplots()
for _, lake_data in lakes_extent_df.iterrows():
    lake_polygon = lake_data.geometry
    ax.plot(*lake_polygon.exterior.xy, c='r')
    for ring in lake_polygon.interiors:
        ax.plot(*ring.xy, c='r')
    ax.text(x=lake_polygon.centroid.x - 4e4,
            y=lake_polygon.centroid.y + 4e4,
            s=lake_data.lake_id)
ax.set_xlim(left=-1.6e6, right=-1.3e6)
ax.set_ylim(bottom=-5.5e5, top=-2.5e5)
ax.set_ylabel('Y  [m]')
ax.set_xlabel('X  [m]')
ctx.add_basemap(
    ax, source=ctx.providers.Esri.WorldImagery, crs=sps_epsg_code,
    attribution=False, zoom=5)

In [ ]:
fig, ax = plt.subplots()
for lake_polygon in lakes_extent_df.geometry:
  ax.plot(*lake_polygon.exterior.xy, c='r')
  for ring in lake_polygon.interiors:
      ax.plot(*ring.xy, c='r')
ax.set_xlim(left=-1.8e6, right=-1.2e6)
ax.set_ylim(bottom=-7e5, top=-1e5)
ax.set_ylabel('Y  [m]')
ax.set_xlabel('X  [m]')
ctx.add_basemap(
    ax, source=ctx.providers.Esri.WorldImagery, crs=sps_epsg_code,
    attribution=False, zoom=5)

## Get the CryoTEMPO-EOLIS gridded data for each subglacial lake

In [ ]:
# in order to query data from Specklia within an area, the corresponding polygon needs to be in EPSG 4326 (longitude-latitude)

lakes_extent_4326_df = lakes_extent_df.to_crs(epsg=4326)

In [ ]:
dataset_name = 'CryoTEMPO-EOLIS Gridded Product'
available_datasets = client.list_datasets()
gridded_product_dataset = available_datasets[
    available_datasets['dataset_name'] == dataset_name].iloc[0]

In [ ]:
lake_gridded_eolis_data = {}
lake_gridded_eolis_sources = {}

for _, lake_data in lakes_extent_4326_df.iterrows():
    query_start_time = perf_counter()
    lake_gridded_eolis_data[lake_data.lake_id], lake_gridded_eolis_sources[lake_data.lake_id] = client.query_dataset(
        dataset_id=gridded_product_dataset['dataset_id'],
        epsg4326_polygon=lake_data.geometry,
        min_datetime=datetime(2011, 1, 1),
        max_datetime=datetime(2023, 9, 1),
        columns_to_return=['timestamp', 'elevation', 'uncertainty'])

    print(f'Query of subglacial lake with ID {lake_data.lake_id} complete in '
          f'{perf_counter()-query_start_time:.2f} seconds, '
          f'{len(lake_gridded_eolis_data[lake_data.lake_id])} points returned, '
          f'drawn from {len(lake_gridded_eolis_sources[lake_data.lake_id])} original sources.')

## Get the CryoTEMPO-EOLIS gridded data for the buffer zone around each subglacial lake

In [ ]:
lakes_with_buffer_extent_df = lakes_extent_df.copy()
# let's use the 'buffer' method to query data around each lake
lakes_with_buffer_extent_df['geometry'] = lakes_extent_df.geometry.buffer(10000)
lakes_with_buffer_extent_4326_df = lakes_with_buffer_extent_df.to_crs(epsg=4326)

In [ ]:
all_gridded_eolis_data = {}
all_gridded_eolis_sources = {}

buffer_gridded_eolis_data = {}

for _, lake_data in lakes_with_buffer_extent_4326_df.iterrows():
    query_start_time = perf_counter()
    all_gridded_eolis_data[lake_data.lake_id], all_gridded_eolis_sources[lake_data.lake_id] = client.query_dataset(
        dataset_id=gridded_product_dataset['dataset_id'],
        epsg4326_polygon=lake_data.geometry,
        min_datetime=datetime(2011, 1, 1),
        max_datetime=datetime(2023, 9, 1),
        columns_to_return=['timestamp', 'elevation', 'uncertainty'])

    print(f'Query of subglacial lake with ID {lake_data.lake_id} complete in '
          f'{perf_counter()-query_start_time:.2f} seconds, '
          f'{len(all_gridded_eolis_data[lake_data.lake_id])} points returned, '
          f'drawn from {len(all_gridded_eolis_sources[lake_data.lake_id])} original sources.')

    # keep all the points that are not within the lake extent (i.e., that were not returned in the first query)
    buffer_gridded_eolis_data[lake_data.lake_id] = pd.concat(
        [all_gridded_eolis_data[lake_data.lake_id], lake_gridded_eolis_data[lake_data.lake_id]]).drop_duplicates(keep=False)

    # perform just a sanity check: the number of points for the buffer zone should match the difference between the second query and the first
    assert len(buffer_gridded_eolis_data[lake_data.lake_id]) == (len(all_gridded_eolis_data[lake_data.lake_id]) - len(lake_gridded_eolis_data[lake_data.lake_id]))
    print(f'The buffer around lake with ID {lake_data.lake_id} '
          f'has {len(buffer_gridded_eolis_data[lake_data.lake_id])} points, as expected.')

## Calculate the elevation change timeseries

We have created two complementary datasets with EOLIS gridded data, one over the lakes and one over the buffer area around the lakes.

First calculate the elevation change separately for each dataset.
Subsequently, derive the net elevation change of the subglacial lakes as the difference between the observed elevation change over the lakes and the average observed elevation change in the area surrounding the lakes.

In [ ]:
def calculate_time_series(gridded_product_data: gpd.GeoDataFrame) -> Tuple[NDArray, NDArray, List[int]]:
    """
    Calculate a weighted average elevation change per month and associated errors.

    Weights are assigned using the CryoTEMPO-EOLIS gridded product uncertainty.

    Parameters
    ----------
    gridded_product_data : gpd.GeoDataFrame
        CryoTEMPO-EOLIS gridded product data.

    Returns
    -------
    Tuple[NDArray, NDArray, List[int]]
        Weighted average elevation change elevation change per month, error on weighted average,
        and dates of each timestep.
    """
    unique_timestamps = sorted(gridded_product_data.timestamp.unique())

    # elevation change is in reference to first gridded product
    reference_dem = gridded_product_data[gridded_product_data.timestamp == unique_timestamps[0]]

    # initialise empty lists for time series data
    weighted_mean_timeseries = []
    weighted_mean_errors = []
    for unique_timestamp in unique_timestamps:
        gridded_product_for_timestamp = gridded_product_data[gridded_product_data.timestamp == unique_timestamp]
        merged_gdf = reference_dem.sjoin(df=gridded_product_for_timestamp, how='inner')
        merged_gdf['elevation_difference'] = merged_gdf['elevation_right'] - merged_gdf['elevation_left']

        # combine uncertainties for elevation difference
        merged_gdf['elevation_difference_unc'] = np.sqrt(
            merged_gdf['uncertainty_right']**2 + merged_gdf['uncertainty_left']**2)

        # calculate average elevation change, weighted by measurement uncertainty
        weighted_mean = (np.sum(merged_gdf['elevation_difference'] / merged_gdf['elevation_difference_unc']**2)
                        / np.sum(1 / merged_gdf['elevation_difference_unc']**2))

        # calculate weighted average uncertainty
        error = np.sqrt(1 / np.sum(1 / merged_gdf['elevation_difference_unc']**2))

        weighted_mean_timeseries.append(weighted_mean)
        weighted_mean_errors.append(error)

    dates = [datetime.fromtimestamp(ts) for ts in unique_timestamps]

    return np.array(weighted_mean_timeseries), np.array(weighted_mean_errors), dates

In [ ]:
timeseries_data = {}
for _, lake_data in lakes_extent_df.iterrows():

    buffer_mean_elevation_change, buffer_uncertainty_mean_elevation_change, buffer_timeseries_dates = calculate_time_series(buffer_gridded_eolis_data[lake_data.lake_id])
    lake_mean_elevation_change, lake_uncertainty_mean_elevation_change, lake_timeseries_dates = calculate_time_series(lake_gridded_eolis_data[lake_data.lake_id])

    # simple sanity check, to ensure that the observations are coincident over the lakes and over the surrounding areas
    assert lake_timeseries_dates == buffer_timeseries_dates

    timeseries_data[lake_data.lake_id] = pd.DataFrame.from_dict({
        'date': lake_timeseries_dates,
        'buffer_mean_elevation_change': buffer_mean_elevation_change,
        'buffer_uncertainty_mean_elevation_change': buffer_uncertainty_mean_elevation_change,
        'lake_mean_elevation_change': lake_mean_elevation_change,
        'lake_uncertainty_mean_elevation_change': lake_uncertainty_mean_elevation_change})
    # calculate the net elevation difference of the subglacial lakes and propagate the uncertainty
    timeseries_data[lake_data.lake_id]['net_mean_elevation_change'] = lake_mean_elevation_change - buffer_mean_elevation_change
    timeseries_data[lake_data.lake_id]['net_uncertainty_mean_elevation_change'] = np.sqrt(lake_uncertainty_mean_elevation_change**2 + buffer_uncertainty_mean_elevation_change**2)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for _, lake_data in lakes_extent_df.iterrows():

    timeseries_df = timeseries_data[lake_data.lake_id]
    ax.plot(timeseries_df.date, timeseries_df.buffer_mean_elevation_change, lw=1, label=lake_data.lake_id)
    ax.fill_between(timeseries_df.date,
                    timeseries_df.buffer_mean_elevation_change - timeseries_df.buffer_uncertainty_mean_elevation_change,
                    timeseries_df.buffer_mean_elevation_change + timeseries_df.buffer_uncertainty_mean_elevation_change,
                    alpha=0.5)

ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Elevation Change in buffer  [m]')
ax.legend(title='Subglacial lake ID')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for _, lake_data in lakes_extent_df.iterrows():

    timeseries_df = timeseries_data[lake_data.lake_id]
    ax.plot(timeseries_df.date, timeseries_df.net_mean_elevation_change, lw=1, label=lake_data.lake_id)
    ax.fill_between(timeseries_df.date,
                    timeseries_df.net_mean_elevation_change - timeseries_df.net_uncertainty_mean_elevation_change,
                    timeseries_df.net_mean_elevation_change + timeseries_df.net_uncertainty_mean_elevation_change,
                    alpha=0.5)

ax.set_xlabel('Date')
ax.set_ylabel('Net cumulative Elevation Change over lakes  [m]')
ax.legend(title='Subglacial lake ID')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for _, lake_data in lakes_extent_df.iterrows():

    timeseries_df = timeseries_data[lake_data.lake_id]
    volume_change = (timeseries_df.net_mean_elevation_change * lake_data.geometry.area) * 1e-9
    uncertainty_volume_change = (timeseries_df.net_uncertainty_mean_elevation_change * lake_data.geometry.area) * 1e-9

    # plot time series
    ax.plot(timeseries_df.date, volume_change, lw=1, label=lake_data.lake_id)
    ax.fill_between(timeseries_df.date,
                    volume_change - uncertainty_volume_change,
                    volume_change + uncertainty_volume_change,
                    alpha=0.5)

ax.set_xlabel('Date')
ax.set_ylabel(r'Cumulative Volume Change [km$^3$]')
ax.legend(title='Subglacial lake ID')
plt.show()